In [328]:
import numpy as np
import cv2
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
import albumentations as a
import warnings
from PIL import Image
import os
from pathlib import Path
from os import listdir
warnings.filterwarnings('ignore')

In [355]:
class Lines:
  def __init__(self):
    return None

  ##### Volume Block
  def volume(self, filepath):
    self.filepath = filepath
    img = cv2.imread(self.filepath)


    ### Detect the Yellow color lines
    gr_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    lower_range = np.array([25,100,20])
    upper_range = np.array([35,255,233])
    maskr = cv2.inRange(gr_img, lower_range, upper_range)


    ### Detect the Green color lines
    lower_range = np.array([40,25,20])
    upper_range = np.array([75,255, 255])
    maskg = cv2.inRange(gr_img, lower_range, upper_range)


    ### Joined the broken Green lines
    kernel = np.ones((1,20), np.uint8)
    d_im = cv2.dilate(maskg, kernel, iterations=2)
    e_img = cv2.erode(d_im, kernel, iterations=3)


    ### Remove the Folders
    !rm -rf "/content/masked_cropped_images"
    !rm -rf "/content/masked_images"


    ### Joined the broken Yellow lines 
    kernel = np.ones((1,20), np.uint8)
    d_im = cv2.dilate(maskr, kernel, iterations=2)
    e_imr = cv2.erode(d_im, kernel, iterations=2)
    if not os.path.exists("/content/masked_images/"):
      os.mkdir("/content/masked_images/")
    cv2.imwrite("/content/masked_images/resultr.png",e_imr)
    cv2.imwrite("/content/masked_images/resultg.png",e_img)
    filepath = "/content/masked_images"


    ### Cropped the images vertically into 20 groups
    for i in os.listdir(filepath):
      if not i.startswith("."):
        input = filepath +"/"+ i
        filename, file_extension = os.path.splitext(input)
        im = Image.open(input)
        imgwidth, imgheight = im.size
        height = imgheight // 1
        width = imgwidth // 20
        for k in range(0, 1):
          for j in range(0, 20):
            box = (j * width, k * height, (j + 1) * width, (k + 1) * height)
            a = im.crop(box)
            if not os.path.exists("/content/masked_cropped_images/"):
              os.mkdir("/content/masked_cropped_images/")
            if not os.path.exists("/content/masked_cropped_images/"+i[:-4]):
              os.mkdir("/content/masked_cropped_images/"+i[:-4])
            a.save("/content/masked_cropped_images/" + i[:-4]+"/" + str(k) + "-" + str(j) + file_extension)


    ### Counted the lines
    filepath = "/content/masked_cropped_images/"
    h1r = 0
    f1r = 0
    for i in os.listdir(filepath):
      h1 = 0
      f1 = 0
      imagepath = "/content/masked_cropped_images/"+i
      for j in os.listdir(imagepath):
        full_ipath = cv2.imread(imagepath+"/"+j,0)
        h,f = cv2.threshold(full_ipath, 80, 255, cv2.THRESH_BINARY_INV)
        contours, hierarchy = cv2.findContours(f, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        lenr = len(contours) - 1
        lenr = 10 - lenr
        if(i == "resultr"):
          h1 = h1+lenr
          h1r = h1
        elif(i == "resultg"):
          f1 = f1+lenr
          f1r = f1


    ### Counted the Volume
    hf = ((((h1r/20)+1)*3)*(((f1r/20)+1)*3))
    print("Volume = ",(hf*0.5)*53.25)


    ### Remove the Folders
    !rm -rf "/content/masked_cropped_images"
    !rm -rf "/content/masked_images"


  ##### Metadata Block
  def metadata(self, filepath):
    self.filepath = filepath
    basename = os.path.basename(self.filepath)
    path = Path(basename).stem
    img = cv2.imread(self.filepath)


    ### Detect the Yellow color lines
    gr_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    lower_range = np.array([25,100,20])
    upper_range = np.array([35,255,233])
    maskr = cv2.inRange(gr_img, lower_range, upper_range)


    ### Detect the Green color lines
    lower_range = np.array([40,25,20])
    upper_range = np.array([75,255, 255])
    maskg = cv2.inRange(gr_img, lower_range, upper_range)
    if not os.path.exists("/content/"+path):
      os.mkdir("/content/"+path)
    if not os.path.exists("/content/"+path+"/masked_images/"):
      os.mkdir("/content/"+path+"/masked_images/")
    if not os.path.exists("/content/"+path+"/masked_images/result"):
      os.mkdir("/content/"+path+"/masked_images/result")
    cv2.imwrite("/content/"+path+"/masked_images/result/yellow.png",maskr)
    cv2.imwrite("/content/"+path+"/masked_images/result/green.png",maskg)
    
    
    ### Joined the broken Green lines
    kernel = np.ones((1,20), np.uint8)
    d_im = cv2.dilate(maskg, kernel, iterations=2)
    e_img = cv2.erode(d_im, kernel, iterations=3)


    ### Joined the broken Yellow lines 
    kernel = np.ones((1,20), np.uint8)
    d_im = cv2.dilate(maskr, kernel, iterations=2)
    e_imr = cv2.erode(d_im, kernel, iterations=2)
    if not os.path.exists("/content/"+path+"/masked_images/"):
      os.mkdir("/content/"+path+"/masked_images/")
    if not os.path.exists("/content/"+path+"/masked_images/joined"):
      os.mkdir("/content/"+path+"/masked_images/joined")
    cv2.imwrite("/content/"+path+"/masked_images/joined/yellow.png",e_imr)
    cv2.imwrite("/content/"+path+"/masked_images/joined/green.png",e_img)
    filepath = "/content/"+path+"/masked_images/joined"


    ### Draw and save the contours of joined images
    for j in os.listdir(filepath):
        img = filepath+"/"+j
        image = cv2.imread(img,0)
        thresh = cv2.threshold(image, 120, 255, cv2.THRESH_BINARY_INV)[1]

        cnts = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cnts = cnts[0] if len(cnts) == 2 else cnts[1]
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
        lines = 0
        for c in cnts:
          cv2.drawContours(image, [c], -1, (0, 255, 0), 3)
          lines += 1
        if not os.path.exists("/content/"+path+"/masked_images/contours"):
          os.mkdir("/content/"+path+"/masked_images/contours")
        cv2.imwrite("/content/"+path+"/masked_images/contours/"+j,image)

    ### Cropped the images vertically into 10 groups
    for i in os.listdir(filepath):
      if not i.startswith("."):
        input = filepath +"/"+ i
        filename, file_extension = os.path.splitext(input)
        im = Image.open(input)
        imgwidth, imgheight = im.size
        height = imgheight // 1
        width = imgwidth // 20
        for k in range(0, 1):
          for j in range(0, 20):
            box = (j * width, k * height, (j + 1) * width, (k + 1) * height)
            a = im.crop(box)
            if not os.path.exists("/content/"+path+"/masked_cropped_images/"):
              os.mkdir("/content/"+path+"/masked_cropped_images/")
            if not os.path.exists("/content/"+path+"/masked_cropped_images/result"):
              os.mkdir("/content/"+path+"/masked_cropped_images/result")
            if not os.path.exists("/content/"+path+"/masked_cropped_images/result/"+i[:-4]):
              os.mkdir("/content/"+path+"/masked_cropped_images/result/"+i[:-4])
            a.save("/content/"+path+"/masked_cropped_images/result/" + i[:-4]+"/" + str(k) + "-" + str(j) + file_extension)


    ### Draw and save the contour lines of cropped images
    filepath = "/content/"+path+"/masked_cropped_images/result/"
    h1r = 0
    f1r = 0
    for i in os.listdir(filepath):
      h1 = 0
      f1 = 0
      imagepath = "/content/"+path+"/masked_cropped_images/result/"+i
      for j in os.listdir(imagepath):
        img = imagepath+"/"+j
        image = cv2.imread(img,0)
        thresh = cv2.threshold(image, 120, 255, cv2.THRESH_BINARY_INV)[1]

        cnts = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cnts = cnts[0] if len(cnts) == 2 else cnts[1]
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
        lines = 0
        for c in cnts:
          cv2.drawContours(image, [c], -1, (0, 255, 0), 3)
          lines += 1
        if not os.path.exists("/content/"+path+"/masked_cropped_images/contour/"):
              os.mkdir("/content/"+path+"/masked_cropped_images/contour/")
        if not os.path.exists("/content/"+path+"/masked_cropped_images/contour/"+i):
              os.mkdir("/content/"+path+"/masked_cropped_images/contour/"+i)
        cv2.imwrite("/content/"+path+"/masked_cropped_images/contour/"+i+"/"+j,image)

L = Lines()

In [356]:
L.metadata("/content/test_image/3638.png")

In [357]:
L.volume("/content/test_image/3638.png")

Volume =  2900.0615625000005


In [358]:
L.metadata("/content/test_image/4446.png")

In [359]:
L.volume("/content/test_image/4446.png")

Volume =  3666.2625


In [360]:
L.metadata("/content/test_image/5659.png")

In [361]:
L.volume("/content/test_image/5659.png")

Volume =  4122.748125


In [362]:
L.metadata("/content/test_image/1819.png")

In [363]:
L.volume("/content/test_image/1819.png")

Volume =  1889.443125


In [364]:
L.metadata("/content/test_image/2223.png")

In [365]:
L.volume("/content/test_image/2223.png")

Volume =  1988.8875


In [366]:
L.metadata("/content/test_image/2829.png")

In [367]:
L.volume("/content/test_image/2829.png")

Volume =  2036.8125
